In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm

# Capstone Project - Function 5

## Function 5 - Chemical Yield

This time you are trying to optimise another four-dimensional black-box. It corresponds to the yield of a chemical process after processing in some factory. This type of process tends to be unimodal. Try to find the combination of chemicals that maximizes the yield!
Given that the process is unimodal (i.e., it has a single peak), an acquisition function that excels in exploitation would be suitable. In this case, Expected Improvement (EI) could be a good choice.

For a unimodal optimization problem like this, the Matérn kernel is often a good choice. The Matérn kernel is less smooth than the RBF kernel, which can make it more suitable for real-world applications where the function may not be infinitely differentiable, such as in this chemical process.

### 1.Loading the available data:

In [3]:
# Loading the baseline data
X = np.load('initial_data/function_5/initial_inputs.npy')
Y = np.load('initial_data/function_5/initial_outputs.npy')
X2 = np.load('initial_data2/function_5/initial_inputs.npy')
Y2 = np.load('initial_data2/function_5/initial_outputs.npy')

X = np.concatenate((X,X2), dtype = float)
Y = np.concatenate((Y,Y2), dtype = float)

# Precision notation
np.set_printoptions(suppress=False)

# Adding knwon inputs/outputs from previous weeks
Xn = np.array([
    [0.241379, 0.862068, 0.862068, 0.896551],
    [0.241379, 0.862068, 0.827586, 0.896551],
    [0.242479, 0.863068, 0.877586, 0.897551],
    [0.241379, 0.862068, 0.896551, 0.896551],    
    [0.271331, 0.822068, 0.765172, 0.865172],
    [0.402855, 0.261167, 0.743289, 0.812927],
    [0.25641,  0.871794, 0.923076, 0.897435],
    [0.25641,  0.871794, 0.923076, 0.923076],
    [0.275862, 0.896551, 0.931034, 0.965517],
    [0.275862, 0.896551, 0.931034, 0.931034],
    [0.294117, 0.911764, 0.941176, 0.970588],
    [0.294117, 0.941176, 0.970588, 0.999999],
    [0.294118, 0.941176, 0.999999, 0.999999],
    [0.294118, 0.970588, 0.999999, 0.999999],
    [0.282051, 0.974353, 0.999999, 0.999999],
    [0.282051, 0.999999, 0.999999, 0.999999],
    [0.282051, 0.974358, 0.999991, 0.999999]
])
X = np.concatenate((X,Xn), dtype = float)

Yn = np.array(
[
    1166.75575,
    1022.52593,
    1251.138603,
    1340.384772,
    574.6109346,
    67.92147931,
    1556.813278,
    1720.323404,
    2275.469664,
    1992.298404,
    2527.55231,
    3401.994446,
    3752.4232057271747,
    4087.948885880335,
    4129.885318,
    4466.835156,
    4129.840243
])
Y = np.concatenate((Y,Yn), dtype = float)

In [4]:
print("X:\n")
print(X)
print("\n")
print("Y:\n")
print(Y)

X:

[[0.19144708 0.03819337 0.60741781 0.41458414]
 [0.75865295 0.53651774 0.65600038 0.36034155]
 [0.43834987 0.8043397  0.21024527 0.15129482]
 [0.70605083 0.53419196 0.26424335 0.48208755]
 [0.83647799 0.19360965 0.6638927  0.78564888]
 [0.68343225 0.11866264 0.82904591 0.56757661]
 [0.55362148 0.66734998 0.32380582 0.81486975]
 [0.35235627 0.32224153 0.11697937 0.47311252]
 [0.15378571 0.72938169 0.42259844 0.44307417]
 [0.46344227 0.63002451 0.10790646 0.9576439 ]
 [0.67749115 0.35850951 0.47959222 0.07288048]
 [0.58397341 0.14724265 0.34809746 0.42861465]
 [0.30688872 0.31687813 0.62263448 0.09539906]
 [0.51114177 0.817957   0.72871042 0.11235362]
 [0.43893338 0.77409176 0.37816709 0.93369621]
 [0.22418902 0.84648049 0.87948418 0.87851568]
 [0.72526172 0.47987049 0.08894684 0.75976022]
 [0.35548161 0.63961937 0.41761768 0.12260384]
 [0.11987923 0.86254031 0.64333133 0.84980383]
 [0.12688467 0.15342962 0.77016219 0.19051811]
 [0.94478251 0.83050925 0.83227839 0.20292205]
 [0.05244

In [5]:
y_max = np.max(Y)
print("Max output until now: ", y_max, "which corresponds to input: ", X[np.where(Y == y_max)][0])

Max output until now:  4466.835156 which corresponds to input:  [0.282051 0.999999 0.999999 0.999999]


In [6]:
# random 
next_query = np.random.uniform(size = 4)
print(next_query)

[0.40842807 0.82929392 0.66434601 0.56469135]


### 2. Build a Bayesian Model

In [13]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C
from scipy.optimize import minimize

# Define the kernel function
kernel_matern = C(1.0, (1e-3, 1e3)) * Matern(length_scale=1.0, nu=1.5)  # Adjust nu for smoothness

# Create a GaussianProcessRegressor object
gpr = GaussianProcessRegressor(kernel=kernel_matern, n_restarts_optimizer=9)

# Fit to data using Maximum Likelihood Estimation of the
gpr.fit(X, Y)

C:\Users\guill\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=9)

### 3. Acquisition Function 1 - Upper Confidence Bound

Upper-Confidence Bound action selection uses uncertainty in the action-value estimates for balancing exploration and exploitation. Since there is inherent uncertainty in the accuracy of the action-value estimates when we use a sampled set of rewards thus UCB uses uncertainty in the estimates to drive exploration.

In [14]:
import itertools as it

x_range = np.linspace(0, 1, 50)

# Create the 4D grid using NumPy
dim = 4
X_grid = np.meshgrid(*([x_range] * dim))
X_grid = np.stack(X_grid, axis=-1).reshape(-1, dim)

mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 1.10 * std

idx_max = np.argmax(ucb1)
UB_NextQuery = X_grid[idx_max]
print("UCB - Next Query - Idea 1: ", UB_NextQuery)

UCB - Next Query - Idea 1:  [0.28571429 1.         1.         1.        ]


### 4. Acquisition Function 2 - Probability of Improvement

In [11]:
# Current best observed value (maximum)
f_best = np.max(Y)

# Compute PI values for candidate points
def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

# Choose the next point with maximum PI value
next_idx = np.argmax(pi_values)
PI_NextQuery = X_grid[next_idx]

print("PI - Next Query - Idea 1: ", PI_NextQuery)

PI - Next Query - Idea 1:  [0.28571429 1.         1.         1.        ]


### 5. Acquisition Function - Thompson Sampling

In [9]:
# Compute Thompson Sampling values for candidate points
def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

# Choose the next point with maximum Thompson Sampling value
next_idx = np.argmax(thompson_values)
TS_NextQuery = X_grid[next_idx]

print("Thompson Sampling - Next Query: ", TS_NextQuery)


Thompson Sampling - Next Query:  [0.29411765 0.97058824 1.         1.        ]


### 6. Acquisition Function - Bayesian Expected Losses

In [10]:
# Compute the expected loss for candidate points
def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - y_max) / sigma
    expected_loss = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss

# We re-use X_grid due to computational needs
expected_loss_values = [compute_expected_loss(x) for x in X_grid]

# Choose the next point with minimum expected loss
next_idx = np.argmin(expected_loss_values)
BL_NextQuery = X_grid[next_idx]

print("Bayesian Expected Loss - Next Query: ", BL_NextQuery)

Bayesian Expected Loss - Next Query:  [0.17647059 0.02941176 0.58823529 0.41176471]


### 7. Acquisition Function - Expected Improvement

In [8]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

# Compute Expected Improvement for candidate points
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

# Choose the next point with maximum EI value
next_idx = np.argmax(ei_values)
EI_NextQuery = X_grid[next_idx]

print("Expected Improvement - Next Query: ", EI_NextQuery)



KeyboardInterrupt



In [12]:
formatted_UCB = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("UCB: ",UB_NextQuery[0],UB_NextQuery[1],UB_NextQuery[2],UB_NextQuery[3], gpr.predict([UB_NextQuery]))
formatted_PI = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("PI: ",PI_NextQuery[0],PI_NextQuery[1], PI_NextQuery[2], PI_NextQuery[3], gpr.predict([PI_NextQuery]))
formatted_TS = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("TS: ",TS_NextQuery[0],TS_NextQuery[1], TS_NextQuery[2], TS_NextQuery[3], gpr.predict([TS_NextQuery]))
formatted_BL = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("BL: ",BL_NextQuery[0],BL_NextQuery[1], BL_NextQuery[2], BL_NextQuery[3], gpr.predict([BL_NextQuery]))
formatted_EI = "{}{:.6f}-{:.6f}-{:.6f}-{:.6f} -> {}".format("EI: ",EI_NextQuery[0],EI_NextQuery[1], EI_NextQuery[2], EI_NextQuery[3], gpr.predict([EI_NextQuery]))

print(formatted_UCB)
print(formatted_PI)
print(formatted_TS)
print(formatted_BL)
print(formatted_EI)

UCB: 0.294118-1.000000-1.000000-1.000000 -> [4041.75681942]
PI: 0.294118-0.970588-1.000000-1.000000 -> [4087.9579414]
TS: 0.294118-0.970588-1.000000-1.000000 -> [4087.9579414]
BL: 0.176471-0.029412-0.588235-0.411765 -> [62.10653175]
EI: 0.294118-1.000000-1.000000-1.000000 -> [4041.75681942]
